## Derived datatypes

In this exercise you will pass data around a ring with a derived datatype instead of an integer or an array like we did so far. You send and receive buffer will be a struct with two integers. 

1. Produce a new datatype that can be used as a buffer with the routines that you have learned in the previous step. 

2. Initialize the struct intigers with `rank` and `10*rank`. Therefore we will pass around two values and calculate two separate sums. 

3. Use the new datatype in the send and receive routine calls. Currently, the data is send with the description `snd_buf, 2, MPI_INTEGER` which you must modify by using a derived datatype and with a type map of “two integers”. 

***
#### C program

In [ ]:
#include <mpi.h>

In [ ]:
%%executable  a.x -- -lmpi

int rank, size;
int i, right, left;

struct buff {
    int i;
    int j;
} snd_buf, rcv_buf, sum;

//__________ send_recv_type; // Produce new send-receive datatype

MPI_Status status;
MPI_Request request;

MPI_Init(NULL, NULL);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &size);

right = (rank+1)      % size;
left  = (rank-1+size) % size;

// Set MPI datatypes for sending and receiving partial sums
//_________________________
//_________________________

// Init
snd_buf.i = rank;
snd_buf.j = 10*rank;
sum.i = 0;
sum.j = 0;

// Modify the send and receive calls to use the derived datatype
for(i = 0; i < size; i++) 
{
    MPI_Issend(&snd_buf, 2, MPI_INT, right, 17, MPI_COMM_WORLD, &request);
    MPI_Recv(&rcv_buf, 2, MPI_INT, left, 17, MPI_COMM_WORLD, &status);
    MPI_Wait(&request, &status);
    snd_buf = rcv_buf;
    sum.i += rcv_buf.i;
    sum.j += rcv_buf.j;
}

printf ("PE%i:\tSum = %i\t%i\n", rank, sum.i, sum.j);

MPI_Finalize();

In [ ]:
!mpirun -np 4 a.x

***
#### Python program

In [ ]:
%%file ring.py
from mpi4py import MPI
from dataclasses import dataclass

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
status = MPI.Status()

right = (rank+1) % size
left = (rank-1+size) % size

@dataclass
class Buff:
    i: int
    j: int

# Produce new send-receive datatype
# Set MPI datatypes for sending and receiving partial sums
#_________________________
#_________________________

# Init
snd_buf = Buff(rank, 10*rank)
rcv_buf = Buff(0, 0)
sum = Buff(0, 0)

# Modify the send and receive calls to use the derived datatype
for i in range (0,size):
    request = comm.issend(snd_buf, dest=right)
    rcv_buf = comm.recv(source=left)
    request.wait()
    snd_buf = rcv_buf
    sum.i += rcv_buf.i
    sum.j += rcv_buf.j

print("PE%i:\tSum = %i\t%i" % (rank, sum.i, sum.j))

In [ ]:
!mpirun -np 4 python ring.py

***
#### Fortran program

In [ ]:
%%file ring.f90
program ring
use mpi

integer ( kind = 4 ) error
integer :: rank, size
integer :: i, right, left

type buff
    sequence
    integer :: i
    integer :: j
end type buff
type(buff), asynchronous :: snd_buf
type(buff) :: rcv_buf, sum

integer :: send_recv_type ! Produce new send-receive datatype
integer :: status(MPI_STATUS_SIZE)
integer :: request

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)

right = mod(rank+1,      size)
left  = mod(rank-1+size, size)

! Set MPI datatypes for sending and receiving partial sums
! _________________________
! _________________________

! Init
snd_buf%i = rank
snd_buf%j = 10*rank
sum%i = 0
sum%j = 0

! Modify the send and receive calls to use the derived datatype
do i = 1, size
    call MPI_Issend(snd_buf, 2, MPI_INTEGER, right, 17, MPI_COMM_WORLD, request, error)
    call MPI_Recv(rcv_buf, 2, MPI_INTEGER, left, 17, MPI_COMM_WORLD, status, error)
    call MPI_Wait(request, status, error)
    if (.NOT.MPI_ASYNC_PROTECTS_NONBLOCKING) call MPI_F_sync_reg(snd_buf)
    snd_buf = rcv_buf
    sum%i = sum%i + rcv_buf%i
    sum%j = sum%j + rcv_buf%j
end do
print *, 'PE', rank, ': Sum =', sum%i, sum%j
call MPI_Finalize(error)
end

In [ ]:
!mpif90 ring.f90 && mpirun -np 4 a.out

***
### You can compare with our solution:

***
#### C solution

In [ ]:
%%executable  a.x -- -lmpi

int rank, size;
int i, right, left;

struct buff {
    int i;
    int j;
} snd_buf, rcv_buf, sum;

MPI_Datatype send_recv_type; // Produce new send-receive datatype

MPI_Status status;
MPI_Request request;

MPI_Init(NULL, NULL);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &size);

right = (rank+1)      % size;
left  = (rank-1+size) % size;

// Set MPI datatypes for sending and receiving partial sums
MPI_Type_contiguous(2, MPI_INT, &send_recv_type);
MPI_Type_commit(&send_recv_type);

// Init
snd_buf.i = rank;
snd_buf.j = 10*rank;
sum.i = 0;
sum.j = 0;

// Modify the send and receive calls to use the derived datatype
for(i = 0; i < size; i++) 
{
    MPI_Issend(&snd_buf, 1, send_recv_type, right, 17, MPI_COMM_WORLD, &request);
    MPI_Recv(&rcv_buf, 1, send_recv_type, left,  17, MPI_COMM_WORLD, &status);
    MPI_Wait(&request, &status);
    snd_buf = rcv_buf;
    sum.i += rcv_buf.i;
    sum.j += rcv_buf.j;
}

printf ("PE%i:\tSum = %i\t%i\n", rank, sum.i, sum.j);

MPI_Finalize();

In [ ]:
!mpirun -np 4 a.x

***
#### Python solution

In [ ]:
%%file ring.py
from mpi4py import MPI
from dataclasses import dataclass

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
status = MPI.Status()

right = (rank+1) % size
left = (rank-1+size) % size

@dataclass
class Buff:
    i: int
    j: int

# Produce new send-receive datatype
# Set MPI datatypes for sending and receiving partial sums
send_recv_type = MPI.INT.Create_contiguous(2)
send_recv_type.Commit()

# Init
snd_buf = Buff(rank, 10*rank)
rcv_buf = Buff(0, 0)
sum = Buff(0, 0)

# Modify the send and receive calls to use the derived datatype
for i in range (0,size):
    request = comm.issend(snd_buf, dest=right)
    rcv_buf = comm.recv(source=left)
    request.wait()
    snd_buf = rcv_buf
    sum.i += rcv_buf.i
    sum.j += rcv_buf.j

print("PE%i:\tSum = %i\t%i" % (rank, sum.i, sum.j))

In [ ]:
!mpirun -np 4 python ring.py

***
#### Fortran solution

In [ ]:
%%file ring.f90
program ring
use mpi

integer ( kind = 4 ) error
integer :: rank, size
integer :: i, right, left

type buff
    sequence
    integer :: i
    integer :: j
end type buff
type(buff), asynchronous :: snd_buf
type(buff) :: rcv_buf, sum

integer :: send_recv_type ! Produce new send-receive datatype
integer :: status(MPI_STATUS_SIZE)
integer :: request

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)

right = mod(rank+1,      size)
left  = mod(rank-1+size, size)

! Set MPI datatypes for sending and receiving partial sums
call MPI_Type_contiguous(2, MPI_INTEGER, send_recv_type, error)
call MPI_Type_commit(send_recv_type, error)

! Init
snd_buf%i = rank
snd_buf%j = 10*rank
sum%i = 0
sum%j = 0

! Modify the send and receive calls to use the derived datatype
do i = 1, size
    call MPI_Issend(snd_buf, 1, send_recv_type, right, 17, MPI_COMM_WORLD, request, error)
    call MPI_Recv(rcv_buf, 1, send_recv_type, left, 17, MPI_COMM_WORLD, status, error)
    call MPI_Wait(request, status, error)
    if (.NOT.MPI_ASYNC_PROTECTS_NONBLOCKING) call MPI_F_sync_reg(snd_buf)
    snd_buf = rcv_buf
    sum%i = sum%i + rcv_buf%i
    sum%j = sum%j + rcv_buf%j
end do
print *, 'PE', rank, ': Sum =', sum%i, sum%j
call MPI_Finalize(error)
end

In [ ]:
!mpif90 ring.f90 && mpirun -np 4 a.out